This script imputes Veteran's Benefits (VB) recipients dollar benefit amount to match the aggregates with United States Department of Veterans Affairs (USVA) statistics for VB. In this current version, we used 2014 CPS data and USVA FY2014 and FY2015 annual reports on VB. Please refer to the documentation in the same folder for more details on methodology and assumptions. The output this script is an individual level dataset that contains CPS personal level ID (PERIDNUM),individual participation indicator (vb_participation, 0 - not a recipient, 1 - current recipient on file, 2 - imputed recipient), and benefit amount.

Input: 2014 CPS (cpsmar2014t.csv), number of recipients and their benefits amount by state in 2014 (VB_administrative.csv)

Output: VB_Imputation.csv

Additional Source links: https://www.va.gov/vetdata/ 

In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import random
import statsmodels.formula.api as sm
import matplotlib.pyplot as plt 

##### Variables we use in Veteran's Benefits

In [2]:
#CPS_dataset = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\cpsmar2014t.csv')
#columns_to_keep = ['hvet_yn','hvetval','fvetval','finc_vet','sur_yn','srvs_val','sur_sc1','sur_val1','tsurval1','sur_sc2',
#                   'sur_val2','tsurval2','vet_val','vet_yn','vet_typ1','vet_typ2','vet_typ3','vet_typ4','vet_typ5','hsur_yn',
#                   'hsurval','fsurval','finc_sur','gestfips','marsupwt','peafever','champ','a_age','wsal_val','semp_val','frse_val',
#                   'pedisdrs', 'pedisear', 'pediseye', 'pedisout', 'pedisphy', 'pedisrem','a_sex','peridnum','fh_seq']
#CPS_dataset = CPS_dataset[columns_to_keep]
#CPS_dataset.to_csv('VB.csv', columns=columns_to_keep, index=False)

In [3]:
CPS_dataset = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\VB information\data\VB.csv')

In [4]:
# VB value
vbvalue = pd.to_numeric(np.where(CPS_dataset.vet_val!= 'None or not in universe', CPS_dataset.vet_val, 0))
# VB indicator
indicator = pd.to_numeric(np.where(CPS_dataset.vet_yn== 'Yes', 1, 0))
# On duty before
active = pd.to_numeric(np.where(CPS_dataset.peafever== 'Yes', 1, 0))

In [5]:
VB = DataFrame(vbvalue.transpose())
VB.columns = ['vbvalue']

In [6]:
# 5 types of VB
VB['kind'] = pd.to_numeric(np.where(CPS_dataset.vet_typ1== 'Yes', 1, 0))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ2== 'Yes', 2, VB.kind))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ3== 'Yes', 3, VB.kind))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ4== 'Yes', 4, VB.kind))
VB.kind = pd.to_numeric(np.where(CPS_dataset.vet_typ5== 'Yes', 5, VB.kind))

In [7]:
VB['indicator'] = indicator
VB['active'] = active
VB['marsupwt'] = CPS_dataset.marsupwt
VB['fh_seq'] = CPS_dataset.fh_seq
VB['gestfips'] = CPS_dataset.gestfips
VB['peridnum'] = CPS_dataset.peridnum

In [8]:
# Prepare income information
wage = pd.to_numeric(np.where(CPS_dataset.wsal_val!= 'None or not in universe', CPS_dataset.wsal_val, 0))
self_employed1 = pd.to_numeric(np.where(CPS_dataset.semp_val!= 'None or not in universe', CPS_dataset.semp_val, 0))
self_employed2 = pd.to_numeric(np.where(CPS_dataset.frse_val!= 'None or not in universe', CPS_dataset.frse_val, 0))
income = wage + self_employed1 + self_employed2
VB['income'] = income

In [9]:
# Prepare age information
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "80-84 years of age",
                             random.randrange(80, 84),
                             CPS_dataset.a_age)
CPS_dataset.a_age = np.where(CPS_dataset.a_age == "85+ years of age",
                             random.randrange(85, 95),
                             CPS_dataset.a_age)
VB['a_age'] = pd.to_numeric(CPS_dataset.a_age)

In [10]:
# Prepare gender inforamtion, 0 for male, 1 for female
VB['sex'] = pd.to_numeric(np.where(CPS_dataset.a_sex == 'Male', 0, 1))

In [11]:
# Prepare disabilty information, 6 types
d1 = np.where(CPS_dataset.pedisdrs == 'Yes',1,0)
d2 = np.where(CPS_dataset.pedisear == 'Yes',1,0)
d3 = np.where(CPS_dataset.pediseye == 'Yes',1,0)
d4 = np.where(CPS_dataset.pedisout == 'Yes',1,0)
d5 = np.where(CPS_dataset.pedisphy == 'Yes',1,0)
d6 = np.where(CPS_dataset.pedisrem == 'Yes',1,0)
VB['d1'] = d1
VB['d2'] = d2
VB['d3'] = d3
VB['d4'] = d4
VB['d5'] = d5
VB['d6'] = d6

In [12]:
# Limitation of potentail recipients
familyactive = VB.groupby('fh_seq')['active'].sum()
factive = DataFrame(familyactive)

In [13]:
# Family active index
VB['familyactive'] = np.zeros(len(VB))
for number in VB.fh_seq:
    VB.familyactive= np.where(VB.fh_seq==number,factive.active[factive.index==number],VB.familyactive)

## Regression model

In [14]:
model = sm.ols(formula='indicator ~ a_age + sex + income + d1 + d2 + d3 + d4 + d5 + d6 + active', data=VB[VB.familyactive>0]).fit()
print model.summary()

                            OLS Regression Results                            
Dep. Variable:              indicator   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     203.4
Date:                Thu, 26 Jan 2017   Prob (F-statistic):               0.00
Time:                        01:11:55   Log-Likelihood:                 1260.6
No. Observations:               18826   AIC:                            -2499.
Df Residuals:                   18815   BIC:                            -2413.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.0110      0.004      2.551      0.0

In [15]:
probs = model.fittedvalues

## Import administrative data

In [16]:
admin = pd.read_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\VB_administrative.csv',
                    dtype={ 'Total VB population': np.float,'Average benefits': np.float, 'Total benefits': np.float, 'Medical care': np.float})
admin.index = admin.Fips

In [17]:
# CPS total benefits and Administrative total benefits
state_benefit = {}
state_recipients = {}
for state in admin.Fips:
    this_state = (VB.gestfips==state)
    CPS_totalb = (VB.vbvalue * VB.marsupwt)[this_state].sum()
    admin_totalb =  admin['Total benefits'][state] 
    CPS_totaln = VB.marsupwt[this_state&VB.indicator==1].sum()
    admin_totaln =  admin["Total veteran receving benefits"][state]

    temp = [admin.State[state], CPS_totalb, admin_totalb, CPS_totaln, admin_totaln]
    state_benefit[state] = temp
    
pre_augment_benefit = DataFrame(state_benefit).transpose()
pre_augment_benefit.columns = ['State', 'CPS total benefits','Admin total benefits',
                               'CPS total individual recipients','Admin total individual recipients']

In [18]:
pre_augment_benefit.to_csv('C:\Users\wangy\OneDrive\Documents\BasicIncomeProject\VB-pre-blow-up.csv')

## Imputation

In [19]:
# caculate difference of SNAP stats and CPS aggregates on recipients number
# by state
diff = {'Fips':[],'Difference in Population':[],'Mean Benefit':[],'CPS Population':[],'VA Population':[]}
diff['Fips'] = admin.Fips
current = (VB.indicator==1)
for FIPS in admin.Fips:
        this_state = (VB.gestfips==FIPS)
        current_tots = VB.marsupwt[current&this_state].sum()
        valid_num = VB.marsupwt[current&this_state].sum() + 0.0000001
        current_mean = ((VB.vbvalue * VB.marsupwt)[current&this_state].sum())/valid_num
        diff['CPS Population'].append(current_tots)
        diff['VA Population'].append(float(admin["Total veteran receving benefits"][admin.Fips == FIPS]))
        diff['Difference in Population'].append(float(admin["Total veteran receving benefits"][admin.Fips == FIPS])- current_tots)
        diff['Mean Benefit'].append(current_mean)

In [20]:
d = DataFrame(diff)
d = d[['Fips', 'Mean Benefit', 'Difference in Population', 'CPS Population', 'VA Population']]
d.to_csv('recipients.csv', index=False)

In [21]:
VB['impute'] = np.zeros(len(VB))
VB['vb_impute'] = np.zeros(len(VB))

non_current = (VB.indicator==0)
current = (VB.indicator==1)
random.seed()

for FIPS in admin.Fips:
    
        print ('we need to impute', d['Difference in Population'][FIPS], 'for state', FIPS)
        
        if d['Difference in Population'][FIPS] < 0:
            continue
        else:
            this_state = (VB.gestfips==FIPS)
            not_imputed = (VB.impute==0)
            pool_index = VB[this_state&not_imputed&non_current].index
            pool = DataFrame({'weight': VB.marsupwt[pool_index], 'prob': probs[pool_index]},
                            index=pool_index)
            pool = pool.sort(columns='prob', ascending=False)
            pool['cumsum_weight'] = pool['weight'].cumsum()
            pool['distance'] = abs(pool.cumsum_weight-d['Difference in Population'][FIPS])
            min_index = pool.sort(columns='distance')[:1].index
            min_weight = int(pool.loc[min_index].cumsum_weight)
            pool['impute'] = np.where(pool.cumsum_weight<=min_weight+10 , 1, 0)
            VB.impute[pool.index[pool['impute']==1]] = 1
            VB.vb_impute[pool.index[pool['impute']==1]] = admin['Average benefits'][FIPS]

        print ('Method1: regression gives', 
                VB.marsupwt[(VB.impute==1)&this_state].sum()) 

('we need to impute', 50892.110000000001, 'for state', 1)
('Method1: regression gives', 50092.869999999995)
('we need to impute', 5614.7399999999998, 'for state', 2)
('Method1: regression gives', 5344.6)
('we need to impute', 49924.690000000002, 'for state', 4)


C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\wangy\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Method1: regression gives', 49032.8)
('we need to impute', 13846.449999999997, 'for state', 5)
('Method1: regression gives', 14010.379999999997)
('we need to impute', 127801.64999999997, 'for state', 6)
('Method1: regression gives', 127784.02999999998)
('we need to impute', 59113.790000000008, 'for state', 8)
('Method1: regression gives', 59427.96000000001)
('we need to impute', 10320.050000000003, 'for state', 9)
('Method1: regression gives', 10120.97)
('we need to impute', 2905.8199999999997, 'for state', 10)
('Method1: regression gives', 3126.84)
('we need to impute', 2460.0299999999997, 'for state', 11)
('Method1: regression gives', 2400.38)
('we need to impute', 101702.34999999992, 'for state', 12)
('Method1: regression gives', 99767.86)
('we need to impute', 29402.470000000001, 'for state', 13)
('Method1: regression gives', 29605.540000000005)
('we need to impute', 9486.9699999999975, 'for state', 15)
('Method1: regression gives', 9296.929999999998)
('we need to impute', 13564.

In [22]:
#Adjustment ratio
results = {}

imputed = (VB.impute == 1)
has_val = (VB.vbvalue != 0)
no_val = (VB.vbvalue == 0)

for FIPS in admin.Fips:
    this_state = (VB.gestfips==FIPS)
    
    current_total = (VB.vbvalue * VB.marsupwt)[this_state].sum() 
    imputed_total = (VB.vb_impute * VB.marsupwt)[this_state&imputed].sum()
    on_file = current_total + imputed_total

    admin_total = admin['Total benefits'][FIPS]
    
    adjust_ratio = admin_total / on_file
    this_state_num = [admin['State'][FIPS], on_file, admin_total, adjust_ratio]
    results[FIPS] = this_state_num
    

    VB.vb_impute = np.where(has_val&this_state, VB.vbvalue * adjust_ratio, VB.vb_impute)
    VB.vb_impute = np.where(no_val&this_state, VB.vb_impute * adjust_ratio, VB.vb_impute)

VB["vb_participation"] = np.zeros(len(VB))
VB["vb_participation"] = np.where(VB.impute==1, 2, 0)#Augmented
VB["vb_participation"] = np.where(has_val, 1, VB.vb_participation)#CPS 


r = DataFrame(results).transpose()
r.columns=['State', 'Imputed', 'Admin', 'adjust ratio']
r.to_csv('amount.csv', index=False)

## Assign Medical care benefit

In [23]:
#Medical care
medical = {}
medicalcare = {}
for FIPS in admin.Fips:
    this_state = (VB.gestfips==FIPS)
    medical[FIPS] = admin['Medical care'][FIPS] / (VB.marsupwt[VB.vb_participation==1][this_state].sum())
    VB.vb_impute = np.where((VB.vb_participation==1) & (this_state), VB.vb_impute + medical[FIPS], VB.vb_impute)
    medicalcare[FIPS] = [admin['State'][FIPS], medical[FIPS]]

In [24]:
VB.to_csv('VB_Imputation.csv', 
         columns=['peridnum','vb_participation', 'vb_impute'],
         index = False)

In [25]:
r = DataFrame(medicalcare).transpose()
r.columns=['State','Individual average medical care']
r.to_csv('medical.csv', index=False)